## 1. Definição do Problema
Uma empresa que oferece empréstimo a pessoas físicas, necessita saber, com base nos dados de seus clientes, quais novos clientes poderão ou não ter acesso ao crédito solicitado (empréstimo).

Pensando no problema da empresa, iremos criar um modelo utilizando *Machine Learning*, com base nas informações históricas dos clientes dessa empresa (cliente que pediram empréstimo que pagaram e que não pagaram), onde o objetivo é com base nesse histórico, prevermos se possíveis novos clientes irão ou não arcar com seus compromissos.

O produto final será uma aplicação web, onde a entrada são os dados do possível novo cliente e com base nesses dados, o modelo irá informar se pode ou não liberar o crédito solicitado.

> **Observação**: *Essa é uma das fases mais importantes de um projeto de Machine Learning (ou de qualquer tipo de projeto), devemos entender a situação, fazer as devidas documentações (análises de requisitos e etc). Se essa fase não for tratada com a devida importância o projeto pode tender a ter problemas futuros.*








## 2. Preparação dos Dados

Importando as bibliotecas padrões que iremos utilizar durante o projeto

In [0]:
import os
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Carregando a Fonte de Dados (Análise de Crédito)
Importando a fonte de dados, aqui por ser um exemplo, será carregado de um arquivo .csv, mas no mundo real pode vir de diversas outros formatos e tipos, como por exemplo de um banco de dados.

# 
*  *.csv (“comma-separated-values” - valores separados por vírgulas)

In [0]:
data = pd.read_csv('loan.csv')

Abaixo, iremos verificar como está dispostos nossos dados, iremos ver as 5 primeiras linhas.

*Podemos observar algumas colunas categóricas (`Gender`, `Married`, `Dependents`, `Education`...), discretas (`ApplicantIncome`) e outras contínuas (`CoapplicantIncome`, `LoanAmount`...).*

In [24]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


Com o comando abaixo, iremos verificar os Quantidade de Linhas e Colunas, Tipos das Colunas, Tamanho entre outras informações.

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               601 non-null object
Married              611 non-null object
Dependents           599 non-null object
Education            614 non-null object
Self_Employed        582 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.4+ KB


Verificando o balanceamento dos Labels (Y - Aprovado, N - Negado), basicamente contamos a quantidade de registros da variável Target: a coluna `Loan_Status`.

Observamos que tem um desbalançeamento das classes...

In [26]:
data.Loan_Status.value_counts()

Y    422
N    192
Name: Loan_Status, dtype: int64

Vamos dar uma equilibrada, reduzindo o valor de Aprovados (Y), vamos pegar apenas 200 (existem diversas outras técnicas para tratar classes desbalançeadas, como podemos ver em [Imbalanced Classification](https://pessoalex.wordpress.com/2019/12/30/imbalanced-classification/)).

In [0]:
data2 = data[data.Loan_Status=='Y'].sample(200)

Abaixo, anexa as 200 amostras da **classe Y** com os registros da **classe N**, em um dataframe que chamamos de **data**

In [0]:
data = data2.append(data[data.Loan_Status=='N'].sample(192))

Verificando novamente o equilíbrio entre as classes.

In [29]:
data.Loan_Status.value_counts()

Y    200
N    192
Name: Loan_Status, dtype: int64

### Checando Missing Values (Valores nulos)
Agora vamos analisar as colunas para ver se encontramos valores nulos.

In [30]:
data2.isnull().sum()

Loan_ID               0
Gender                6
Married               0
Dependents            3
Education             0
Self_Employed        12
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            6
Loan_Amount_Term      5
Credit_History       18
Property_Area         0
Loan_Status           0
dtype: int64

Podemos observar que temos algumas colunas contendo valores nulos, como por exemplo: `Gender`, `Married`, `Dependents`, `Credit_History` e etc.

Existem algumas técnicas para preenchimento de valores nulos, no nosso projeto, iremos utilizar dois tipos:
- Valor Majoritário (assume o valor majoritário da variável e adiciona nos registros nulos;
- Valor Médio (calcula a média dos valores da variável e adiciona nos registros nulos).

Preenchendo Missing Values:

- `Dependents`: Assumindo o valor majoritário.
- `Self_Employed`: Assumindo o valor majoritário.
- `Loan_Amount_Term`: Preenchendo com o valor médio.
- `Credit_History`: Assumindo o valor majoritário.
- `Married`: Assumindo o valor majoritário.
- `Gender`: Assumindo o valor majoritário.

Seguem os comandos...

In [0]:
data['Gender'] = data['Gender'].fillna('Male')

In [0]:
data['Married'] = data['Married'].fillna('No')

In [0]:
data['Dependents'] = data['Dependents'].fillna('0')

In [0]:
data['Self_Employed'] = data['Self_Employed'].fillna('No')

In [0]:
data['LoanAmount'] = data['LoanAmount'].fillna(data['LoanAmount'].mean())

In [0]:
data['Credit_History'] = data['Credit_History'].fillna(1.0)

In [0]:
data['Loan_Amount_Term'] = data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mean())

#### Checando novamente Missing Values
Após a execução dos comandos acima, vamos verificar novamente se as colunas contém valores nulos.

*Veremos no resultado do comando abaixo que não temos mais valores nulos em nenhuma das colunas.*

In [38]:
data.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

### Transformando dados categóricos

Antes de continuar, vamos só contextualizar alguns termos, que já usamos anteriormente e iremos usar a seguir:

**O que são variávies categóricas?** É uma variável estatística, medida em uma escala nominal, cujas categorias identifiquem a sociedade da classe ou de grupo. ([Fonte](https://pt.wikipedia.org/wiki/Vari%C3%A1vel_categ%C3%B3rica))

**O que são variávies discretas e contínuas?** 

**Dicretas** quando o conjunto de resultados possíveis é finito ou enumerável. Exemplo: número de filhos, alunos numa escola etc.

**Contínuas** quando os valores são expressos como intervalo ou união de números reais. Exemplo: peso, massa, altura, pressão sistólica, idade , nível de açúcar no sangue.

([Fonte](https://pt.wikipedia.org/wiki/Vari%C3%A1vel_(estat%C3%ADstica)))

---

Várias colunas do dataframe (data) são categóricas, precisamos transforma-las em discretas ("discretiza-las"), são elas: `Gender`, `Married`, `Education`, `Self_Employed`, `Dependents` e `Loan_Status`.

Existem algumas bibliotecas que fazem esse mapeamento, como por exemplo: [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) e [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder), mas aqui, por serem poucas iremos mapear na mão mesmo até pra ter um entendimento maior do que se trata esse processo de transformar dados categóricos em discretos.

Realizando os mapeamentos e substituindo os valores das colunas no dataframe (data)

In [0]:
gender_values = {'Female' : 0, 'Male' : 1} 
married_values = {'No' : 0, 'Yes' : 1}
education_values = {'Graduate' : 0, 'Not Graduate' : 1}
employed_values = {'No' : 0, 'Yes' : 1}
dependent_values = {'3+': 3, '0': 0, '2': 2, '1': 1}
loan_values = {'Y':1,'N':0}
data.replace({'Gender': gender_values,
                 'Married': married_values, 
                 'Education': education_values,
                 'Self_Employed': employed_values, 
                 'Dependents': dependent_values,
                 'Loan_Status': loan_values
                }, inplace=True)

### Seleção de *Features*
A seleção de atributos (colunas) é uma fase bastante importante para o projeto de Machine Learning, saber quais atributos utilizar para gerar um melhor ganho para seu modelo é visto como uma arte! Esse processo exige diversos testes e simulações.

Existem também ferramentas e técnicas (como por exemplo, correlação entre variáveis e etc) que podem ajudar nisso, aqui iremos pelo conhecimento do conjunto de dados utilizado e iremos excluir as seguintes features (atributos): `Loan_ID`,`CoapplicantIncome`,`Loan_Amount_Term`,`Credit_History`,`Property_Area` esses atributos não agregam muito para a criação do modelo.



Executando o comando drop para realizar essa exclusão

In [0]:
data.drop(['Loan_ID','CoapplicantIncome','Loan_Amount_Term','Credit_History','Property_Area'],axis=1,inplace=True)

Vamos verificar como ficou o dataframe após a nossa seleção de atributos (*features*)

In [42]:
data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Status
427,1,1,2,0,0,5935,133.0,1
124,1,1,0,1,0,4300,194.0,1
213,1,1,3,1,1,5703,130.0,1
79,1,1,3,1,1,3333,130.0,1
263,0,0,0,0,0,7200,120.0,1


Pronto, agora todas as colunas (features) são discretas ou contínuas...

Vamos agora para a criação do modelo.

## 3. Criando o Modelo de *Machine Learning*

Um modelo de *machine learning* é um arquivo que foi treinado para reconhecer determinados tipos de padrões. Você treina um modelo em um conjunto de dados, fornecendo a ele um algoritmo que pode ser usado para ponderar e aprender com esses dados. ([Fonte](https://docs.microsoft.com/pt-br/windows/ai/windows-ml/what-is-a-machine-learning-model))

Utilizaremos a biblioteca scikit-learn (originalmente scikits.learn) é uma biblioteca de aprendizado de máquina de código aberto para a linguagem de programação Python.Ela inclui vários algoritmos de classificação, regressão e agrupamento incluindo máquinas de vetores de suporte, florestas aleatórias, gradient boosting, k-means e DBSCAN, e é projetada para interagir com as bibliotecas Python numéricas e científicas NumPy e SciPy. ([Fonte](https://pt.wikipedia.org/wiki/Scikit-learn))

Esse problema que estamos tratando é um **Problema de Classificação**, e iremos utilizar o algoritmo *Random Forest*.

Essa fase também exige bastante testes de classificadores para encontrar aquele que melhor se enquadra em seus dados.


Abaixo importamos as bibliotecas do algoritmo mencionado acima e também importamos uma biblioteca que divide o modelo em conjunto de treinamento e teste (*train_test_split*)

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

Instanciando o classificador *Random Forest*

In [0]:
clf_rf = RandomForestClassifier(n_estimators=100,min_samples_split=2)

Dividindo o conjunto de dados (Valores - X_train (variáveis independentes) e Rótulos - y (*target* ou variávies dependentes))

In [0]:
X_train = data.drop('Loan_Status',axis=1)
y = data['Loan_Status']

Agora, vamos dividir o conjunto em Treinamento e Teste, utilizando o *train_test_split* importado acima (tamanho do teste: 20% do conjunto)

In [0]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X_train, y,test_size=0.20,random_state=42)

**Treinando o Modelo** - Na etapa abaixo o algoritmo aprenderá os padrões de dados de treinamento que mapeiam as variáveis para o destino e gerará um modelo que captura esses relacionamentos. O modelo de *Machine Learning* poderá, então, ser usado para obter previsões dos novos dados cuja resposta de destino você não conhece.

O modelo é o **clr_rf**.

In [0]:
clf_rf.fit(X_treino,y_treino)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### Métricas de Validação ###
Uma outra fase muito importante é a fase de validação, onde veremos o quão bom é nosso modelo gerado na fase anterior.

Importando biblioteca de métricas do *sklearn*

In [0]:
from sklearn import metrics

Iremos utilizar o **crosstab** para visualizar as classificações.

Onde veremos a proporção de acertos, comparando o que foi **Predito** com o **Real**.

In [0]:
print (pd.crosstab(y_teste, clf_rf.predict(X_teste), rownames=['Real'], colnames=['Predito'], margins=True), '')

(Predito   0   1  All
Real                
0        17  11   28
1        25  26   51
All      42  37   79, '')


Vamos agora gerar um **Relatório de Classificação**, ele nos mostra com mais detalhes algumas métricas importantes, tais como: *precision, recall, f1-score*...

In [0]:
print (metrics.classification_report(y_teste,clf_rf.predict(X_teste)))

              precision    recall  f1-score   support

           0       0.40      0.61      0.49        28
           1       0.70      0.51      0.59        51

   micro avg       0.54      0.54      0.54        79
   macro avg       0.55      0.56      0.54        79
weighted avg       0.60      0.54      0.55        79



## 4. Colocando em Produção
Aqui iremos colocar pra funcionar o nosso modelo, iremos criar uma página da web bem simples para realizar os devidos testes.

### Persistindo o modelo de Machine Learning para o disco.

Antes, vamos persistir nosso modelo para o disco! Utilizaremos a biblioteca **joblib** (que importaremos a seguir)

Para que estou fazendo isso? Para mandar o modelo persistido para meu servidor web.

In [0]:
from sklearn.externals import joblib

Persistindo o modelo em disco com o comando *dump* do **joblib**.

In [0]:
joblib.dump(clf_rf, 'model/model.pkl')

['/content/drive/My Drive/0. Business/2. Consultoria em Dados/2. IA, ML/0. Scripts, Exemplos, Cursos/Scripts exemplos/Deploy em Produc\xcc\xa7a\xcc\x83o/model.pkl']

#### Carregando o modelo a partir do disco para a memória no servidor Web

In [0]:
model = joblib.load('model.pkl')

Vamos verificar os atributos do modelo...

In [0]:
print("Atributos do Modelo:\n\nClasses:{}\n\nEstimators:{}\n\nParametros:{}".format(model.classes_,model.n_estimators,model.base_estimator))

Atributos do Modelo:

Classes:[0 1]

Estimators:100

Parametros:DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


Tudo OK, agora vamos testar!

### Teste de Classificação
Vamos criar um teste, como se fosse a entrada de dados da nossa página da Web (imagem no final do arquivo)

Na página devemos informar algumas informações dos possíveis clientes, como: 
- Sexo (Masculino(1) e Feminino(0))
- Número de Dependentes
- Casado (Sim(1) ou Não(0))
- Grau de Instrução (Graduado(1) ou Não Graduado(0))
- Trabalha por conta própria (Sim(1) ou Não(2))
- Rendimento (rendimentos do cliente)
- Valor do Emprestimo (valor que o cliente quer emprestado)

A ordem que vai para o modelo é: Gender(Sexo), Married (Casado), Dependents (Número de Dependentes), Education (Grau de Instrução), Self_Employed (Trabalha por conta própria), ApplicantIncome (Rendimentos) e	LoanAmount (Valor do Emprestimo)

Realizando um testes com as seguintes informações:
- Gender(Sexo) = 1
- Married (Casado) = 1
- Dependents (Número de Dependentes) = 3
- Education (Grau de Instrução) = 0
- Self_Employed (Trabalha por conta própria) = 0
- ApplicantIncome (Rendimentos) = 9504
- LoanAmount (Valor do Emprestimo) = 275

In [0]:
teste = np.array([[1,1,3,0,0,9504,275.0]])

Rondando o Modelo pra prever o teste que fizemos:

In [0]:
model.predict(teste)

array([0])

O Modelo nos retornou as seguintes probabilidades:

In [0]:
model.predict_proba(teste)

array([[0.56, 0.44]])

Ou seja, o modelo informou com base nos dados de treinamento, que o cliente (do teste) tem a probabilidade de 56% de pagar o empréstimo, então ele irá liberar o crédito solicitado para esse cliente.

### Publicando na Web
Estrutura do Diretório
- Model
 - modelo gerado acima deve ficar nessa pasta (model.pkl)
- static
 - arquivos auxiliares da página
- templates
 - página principal (template.html)
- *srv.py*
 - Renderizando a página usando Flask

#### Publicando...
No terminal, execute:

```
C:\Python>python srv.py

 * Serving Flask app "srv" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://localhost:5500/ (Press CTRL+C to quit)

```

##### Página Publicada
![alt text](https://pessoalex.files.wordpress.com/2020/02/producao.png)





## Versionamento
- **v1.0** - Focado no detalhamento do processo de um Projeto de *Machine Learning* (em 16/02/2020);


## Referências
- Fases do Projeto ([Mauro Filho](http://cursodedatascience.com/))
- Base ([Minerando Dados](https://minerandodados.com.br/))
- Classificador: Random Forest Classifier ([Link](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html))
- Model Persistence ([joblib](https://scikit-learn.org/stable/modules/model_persistence.html))
- Modelo simples de Desenvolvimento Web ([Flask](https://www.treinaweb.com.br/blog/o-que-e-flask/))
- Alex Souza ([Blog](https://pessoalex.wordpress.com/))
